<h1 align='center'>Data Exploratory Analysis For Credit Card Data</h1>

# Business Problem:

<b>In order to effectively produce quality decisions in the modern credit card industry, knowledge must be gained through effective data analysis and modelling. Through the use of dynamic data-driven decision-making tools and procedures, information can be gathered to successfully evaluate all aspects of credit card operations. PSPD Bank has banking operations in more than 50 countries across the globe. Mr. Jim Watson, CEO, wants to evaluate areas of bankruptcy, fraud and collections, respond to customer requests for help with proactive offers and services. </b><br><br>
<b>Following are some of Mr. Watson's questionsn to understand the customer spend and repayment behaviour</b>

# Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os

# Import the datasets

In [ ]:

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
customer = pd.read_csv("/kaggle/input/credit-card-exploratory-data-analysis/Customer Acqusition.csv",usecols=["Customer","Age","City","Product","Limit","Company","Segment"])
repay = pd.read_csv("/kaggle/input/credit-card-exploratory-data-analysis/Repayment.csv",usecols = ["Customer","Month","Amount"])
spend = pd.read_csv("/kaggle/input/credit-card-exploratory-data-analysis/spend.csv",usecols=["Customer","Month","Type","Amount"])

In [ ]:
customer.head()

In [ ]:
repay.head()

In [ ]:
spend.head(2)

# Exploratory Data Analysis

In [ ]:
print(customer.shape)
print(repay.shape)
print(spend.shape)

In [ ]:
customer.dtypes

In [ ]:
repay.dtypes

In [ ]:
spend.dtypes

In [ ]:
spend.isnull().sum()

In [ ]:
customer.isnull().sum()

In [ ]:
repay.isnull().sum()

In [ ]:
# dropping null values present in 'repay' data set
repay.dropna(inplace=True)

In [ ]:
repay.isnull().sum()

# (1) In the above dataset,

## (a) In case age is less than 18, replace it with mean of age values.

In [ ]:
mean_original = customer["Age"].mean()

In [ ]:
print("The mean of Age column is",mean_original)

In [ ]:
#replacing age less than 18 with mean of age values
customer.loc[customer["Age"] < 18,"Age"] = customer["Age"].mean()

In [ ]:
mean_new = customer["Age"].mean()

In [ ]:
print("The new mean of Age column is",mean_new)

In [ ]:
customer.loc[customer["Age"] < 18,"Age"]

In [ ]:
print("All the customers who have age less than 18 have been replaced by mean of the age column.")

##  (b) In case spend amount is more than the limit, replace it with 50% of that customer’s limit. (customer’s limit provided in acquisition table is the per transaction limit on his card)

In [ ]:
customer.head(2)

In [ ]:
spend.head(2)

In [ ]:
#merging customer and spend table on the basis of "Customer" column
customer_spend = pd.merge(left=customer,right=spend,on="Customer",how="inner")

In [ ]:
customer_spend.head()

In [ ]:
customer_spend.shape

In [ ]:
#all the customers whose spend amount is more than the limit,replacing with 50% of that customer’s limit
customer_spend[customer_spend["Amount"] > customer_spend['Limit']]

In [ ]:
#if customer's spend amount is more than the limit,replacing with 50% of that customer’s limit
customer_spend.loc[customer_spend["Amount"] > customer_spend["Limit"],"Amount"] = (50 * customer_spend["Limit"]).div(100)

In [ ]:
#there are no customers left whose spend amount is more than the limit
customer_spend[customer_spend["Amount"] > customer_spend['Limit']]

## (c)  Incase the repayment amount is more than the limit, replace the repayment with the limit.

In [ ]:
customer.head(1)

In [ ]:
repay.head(1)

In [ ]:
#merging customer and spend table on the basis of "Customer" column
customer_repay = pd.merge(left=repay,right=customer,on="Customer",how="inner")

In [ ]:
customer_repay.head()

In [ ]:
#all the customers where repayment amount is more than the limit.
customer_repay[customer_repay["Amount"] > customer_repay["Limit"]]

In [ ]:
#customers where repayment amount is more than the limit, replacing the repayment with the limit.
customer_repay.loc[customer_repay["Amount"] > customer_repay["Limit"],"Amount"] = customer_repay["Limit"]

In [ ]:
#there are no customers left where repayment amount is more than the limit.
customer_repay[customer_repay["Amount"] > customer_repay["Limit"]]

# (2) From the above dataset create the following summaries:

## (a) How many distinct customers exist?

In [ ]:
distinct_customers = customer["Customer"].nunique()

In [ ]:
print("Number of distinct customers are",distinct_customers)

## (b) How many distinct categories exist?

In [ ]:
#customers from different segments
customer["Segment"].value_counts()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot('Segment',data=customer)
plt.show()

In [ ]:
print("We can see from the countplot that number of distinct categories are", len(customer["Segment"].value_counts()))

## (c) What is the average monthly spend by customers?

In [ ]:
spend.head()

In [ ]:
#converting Month column of "spend" table to date time format
spend['Month'] = pd.to_datetime(spend['Month'])

In [ ]:
spend.head()

In [ ]:
#creating new columns which show "Month" and "Year"
spend['Monthly'] = spend['Month'].apply(lambda x:pd.Timestamp.strftime(x,format="%B"))
spend['Yearly'] = spend['Month'].apply(lambda x:pd.Timestamp.strftime(x,format="%Y"))

In [ ]:
spend.head()

In [ ]:
#grouping the dataset based on 'Yearly' and 'monthly'
customer_spend_group= round(spend.groupby(['Yearly','Monthly']).mean(),2)

In [ ]:
customer_spend_group

##  (d) What is the average monthly repayment by customers?

In [ ]:
repay.head(2)

In [ ]:
#coverting "Month" column to date time format
repay["Month"] = pd.to_datetime(repay["Month"])

In [ ]:
repay.head(2)

In [ ]:
repay.dtypes

In [ ]:
#creating new columns which show "Month" and "Year"
repay['Monthly'] = repay['Month'].apply(lambda x:pd.Timestamp.strftime(x,format="%B"))
repay['Yearly'] = repay['Month'].apply(lambda x:pd.Timestamp.strftime(x,format="%Y"))

In [ ]:
#grouping the dataset based on 'Yearly' and 'monthly'
customer_repay_group= round(repay.groupby(['Yearly','Monthly']).mean(),2)

In [ ]:
customer_repay_group

## (e)  If the monthly rate of interest is 2.9%, what is the profit for the bank for each month? 

In [ ]:
#merging all the three tables. Alreaady merged customer and spend table in 'customer_spend'. Using "customer_spend" and "repay"
#table to form the final "customer_spend_repay" table
customer_spend_repay = pd.merge(left=customer_spend,right=repay,on="Customer",how="inner")

In [ ]:
customer_spend_repay.head(2)

In [ ]:
# renaming the columns for clearity
customer_spend_repay.rename(columns={"Amount_x":"Spend_Amount","Amount_y":"Repay_Amount"},inplace=True)

In [ ]:
customer_spend_repay.head()

In [ ]:
# grouping the data based on "Yearly","Month_x" columns to get the 'Spend_Amount'and 'Repay_Amount'
interest_group = customer_spend_repay.groupby(["Yearly","Monthly"])['Spend_Amount','Repay_Amount'].sum()

In [ ]:
interest_group

In [ ]:
 # Monthly Profit = Monthly repayment – Monthly spend.
interest_group['Monthly Profit'] = interest_group['Repay_Amount'] - interest_group['Spend_Amount']

In [ ]:
interest_group

In [ ]:
#interest earned is 2.9% of Monthly Profit
interest_group['Interest Earned'] = (2.9* interest_group['Monthly Profit'])/100

In [ ]:
interest_group

## (f) What are the top 5 product types?

In [ ]:
spend.head()

In [ ]:
#top 5 product types on which customer is spending
spend['Type'].value_counts().head()

In [ ]:
spend['Type'].value_counts().head(5).plot(kind='bar')
plt.show()

## (g)  Which city is having maximum spend?

In [ ]:
customer_spend.head()

In [ ]:
city_spend = customer_spend.groupby("City")["Amount"].sum().sort_values(ascending=False)

In [ ]:
city_spend

In [ ]:
plt.figure(figsize=(5,10))
city_spend.plot(kind="pie",autopct="%1.0f%%",shadow=True,labeldistance=1.0,explode=[0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
plt.title("Amount spent on credit card by customers from different cities")
plt.show()

In [ ]:
print("From above pie chart we can see that Cochin is having maximum spend.")

## (h) Which age group is spending more money?

In [ ]:
#creating new column "Age Group" with 8 bins between 18 to 88 
customer_spend["Age Group"] =  pd.cut(customer_spend["Age"],bins=np.arange(18,88,8),labels=["18-26","26-34", "34-42" ,"42-50" ,"50-58","58-66","66-74","74-82"],include_lowest=True)

In [ ]:
customer_spend.head()

In [ ]:
#grouping data based on "Age Group" and finding the amount spend by each age group and arranging in descending oreder
age_spend = customer_spend.groupby("Age Group")['Amount'].sum().sort_values(ascending=False)

In [ ]:
age_spend

In [ ]:
plt.figure(figsize=(5,10))
age_spend.plot(kind = "pie",autopct="%1.0f%%",explode=[0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0],shadow=True)
plt.show()

In [ ]:
print("From the pie chart shown above we can say that age group 42 - 50 is spending more money")

## (i) Who are the top 10 customers in terms of repayment?

In [ ]:
customer_repay.head()

In [ ]:
#grouping based on "Customer" column to find top 10 customers
customer_repay.groupby("Customer")[["Amount"]].sum().sort_values(by="Amount",ascending=False).head(10)

## (3) Calculate the city wise spend on each product on yearly basis. Also include a graphical representation for the same.

In [ ]:
customer_spend.head()

In [ ]:
#converting "Month" column to date time 
customer_spend["Month"] = pd.to_datetime(customer_spend["Month"])

In [ ]:
#creating new column "year" 
customer_spend['Year'] = customer_spend['Month'].apply(lambda x:pd.Timestamp.strftime(x,format="%Y"))

In [ ]:
customer_spend.head(2)

In [ ]:
customer_spend_pivot = pd.pivot_table(data = customer_spend,index=["City","Year"],columns='Product',aggfunc="sum",values="Amount")

In [ ]:
customer_spend_pivot

In [ ]:
customer_spend_pivot.plot(kind="bar",figsize=(18,5),width=0.8)
plt.ylabel("Spend Amount")
plt.title("Amount spended by customers according to year and city")
plt.show()

# (4) Create graphs for
 

## (a) Monthly comparison of total spends, city wise

In [ ]:
customer_spend.head()

In [ ]:
#creating new column "Monthly" 
customer_spend['Monthly'] = customer_spend['Month'].apply(lambda x:pd.Timestamp.strftime(x,format="%B"))

In [ ]:
customer_spend.head()

In [ ]:
#grouping data based on "Monthly" and "City" columns
month_city = customer_spend.groupby(["Monthly","City"])[["Amount"]].sum().sort_index().reset_index()

In [ ]:
#creating pivot table based on "Monthly" and "City" columns
month_city =pd.pivot_table(data=customer_spend,values='Amount',index='City',columns='Monthly',aggfunc='sum')

In [ ]:
month_city

In [ ]:
month_city.plot(kind="bar",figsize=(18,6),width=0.8)
plt.show()

## (b) Comparison of yearly spend on air tickets

In [ ]:
customer_spend.head()

In [ ]:
air_tickets = customer_spend.groupby(["Year","Type"])[["Amount"]].sum().reset_index()

In [ ]:
filtered = air_tickets.loc[air_tickets["Type"]=="AIR TICKET"]

In [ ]:
filtered

In [ ]:
plt.bar(filtered["Year"],height=filtered["Amount"],color="orange")
plt.xlabel("Year")
plt.ylabel("Amount Spent")
plt.title("Comparison of yearly spend on air tickets")
plt.show()


## (c)  Comparison of monthly spend for each product (look for any seasonality that exists in terms of spend)

In [ ]:
customer_spend.head(2)

In [ ]:
#creating pivot table based on "Monthly" and "Product" columns
product_wise = pd.pivot_table(data=customer_spend,index='Product',columns='Monthly',values='Amount',aggfunc='sum')

In [ ]:
product_wise

In [ ]:
product_wise.plot(kind="bar",figsize=(18,6),width=0.8)
plt.ylabel("Amount Spend")
plt.title("Amount spent monthly on different products")
plt.show()

<b>We can see from the above graph that the sales are high for all the Products during the months:</b>
<ul>
    <li>January</li>
    <li>February</li>
    <li>March</li>
    <li>April</li>
    <li>May</li></ul>
<b> Out of these months,highest sales are in January </b> 

## (5) Write user defined PYTHON function to perform the following analysis: You need to find top 10 customers for each city in terms of their repayment amount by different products and by different time periods i.e. year or month. The user should be able to specify the product (Gold/Silver/Platinum) and time period (yearly or monthly) and the function should automatically take these inputs while identifying the top 10 customers.

In [ ]:
customer_repay.head(2)

In [ ]:
# converting 'Month' column to date time format
customer_repay['Month'] = pd.to_datetime(customer_repay['Month'])

In [ ]:
#creating new column "Monthly" and "Yearly" using already existing 'Month' column
customer_repay['Monthly'] = customer_repay['Month'].apply(lambda x:pd.Timestamp.strftime(x,format="%B"))
customer_repay['Yearly'] = customer_repay['Month'].apply(lambda x:pd.Timestamp.strftime(x,format="%Y"))

In [ ]:
def summary_report(product,timeperiod):
    print('Give the product name and timeperiod for which you want the data')
    if product.lower()=='gold' and timeperiod.lower()=='monthly':
        pivot = customer_repay.pivot_table(index=['Product','City','Customer'],columns='Monthly',aggfunc='sum',values='Amount')
        result = pivot.loc[('Gold',['BANGALORE','COCHIN','CALCUTTA','BOMBAY','CHENNAI','TRIVANDRUM','PATNA','DELHI']),:]
    elif product.lower()=='gold' and timeperiod.lower()=='yearly':
        pivot = customer_repay.pivot_table(index=['Product','City','Customer'],columns='Yearly',aggfunc='sum',values='Amount')
        result = pivot.loc[('Gold',['BANGALORE','COCHIN','CALCUTTA','BOMBAY','CHENNAI','TRIVANDRUM','PATNA','DELHI']),:]
    elif product.lower()=='silver' and timeperiod.lower()=='monthly':
        pivot = customer_repay.pivot_table(index=['Product','City','Customer'],columns='Monthly',aggfunc='sum',values='Amount')
        result = pivot.loc[('Silver',['BANGALORE','COCHIN','CALCUTTA','BOMBAY','CHENNAI','TRIVANDRUM','PATNA','DELHI']),:]
    elif product.lower()=='silver' and timeperiod.lower()=='yearly':
        pivot = customer_repay.pivot_table(index=['Product','City','Customer'],columns='Yearly',aggfunc='sum',values='Amount')
        result = pivot.loc[('Silver',['BANGALORE','COCHIN','CALCUTTA','BOMBAY','CHENNAI','TRIVANDRUM','PATNA','DELHI']),:]
    if product.lower()=='platinum' and timeperiod.lower()=='monthly':
        pivot = customer_repay.pivot_table(index=['Product','City','Customer'],columns='Monthly',aggfunc='sum',values='Amount')
        result = pivot.loc[('Platinum',['BANGALORE','COCHIN','CALCUTTA','BOMBAY','CHENNAI','TRIVANDRUM','PATNA','DELHI']),:]
    elif product.lower()=='platinum' and timeperiod.lower()=='yearly':
        pivot = customer_repay.pivot_table(index=['Product','City','Customer'],columns='Yearly',aggfunc='sum',values='Amount')
        result = pivot.loc[('Platinum',['BANGALORE','COCHIN','CALCUTTA','BOMBAY','CHENNAI','TRIVANDRUM','PATNA','DELHI']),:]
    return result

In [ ]:
summary_report('gold','monthly')